# 2.6. 부록: 텐서플로우 텐서

**참고**

[Introduction to Tensors](https://www.tensorflow.org/guide/tensor)를 요약, 정리한 내용입니다.

In [1]:
import tensorflow as tf
import numpy as np

## 상수 텐서와 변수 텐서

텐서플로우는 두 종류의 텐서 자료형을 지원한다.

- `tf.Tensor` 자료형
    - 상수 텐서
    - 입출력 데이터 등 변하지 않는 값을 다룰 때 사용.
    - 불변 자료형
- `tf.Variable` 자료형
    - 변수 텐서
    - 모델의 가중치, 편향 등 항목의 업데이트가 필요할 때 사용되는 텐서.
    - 가변 자료형

사용법은 기본적으로 넘파이 어레이와 유사하며, GPU 연산과 그레이디언트 자동계산 등
신경망 모델 훈련에 최적화된 기능을 제공한다.


### 상수 텐서

상수 텐서를 다양한 방식으로 생성할 수 있다.
아래 코드는 직접 상수 텐서의 모양과 항목을 지정한다.

In [2]:
x = tf.constant([[1., 2.], [3., 4.]])
print(x)

tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)


넘파이 어레이의 경우와 거의 유사하게 다양한 텐서를 생성하는 함수들이 제공된다.
`tf.ones()` 함수는 1로만 이루어진 상수 텐서를 생성한다.

In [3]:
x = tf.ones(shape=(2, 1))
print(x)

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)


반면에 `tf.zeros()` 텐서는 0으로만 이루어진 상수 텐서를 생성한다.

In [4]:
x = tf.zeros(shape=(2, 1))
print(x)

tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


확률적 통계 분포를 따르는 항목으로 구성된 상수 텐서를 생성하는 함수는 딥러닝에서 매우 유용하다.

- `normal()` 함수: 평균값(`mean`)과 표준편차(`stddev`)를 지정하면 해당 정규분포를 따르는 항목들을 이용하여
원하는 모양의 상수 텐서를 생성한다.

In [5]:
x = tf.random.normal(shape=(3, 1), mean=0., stddev=1.)
print(x)

tf.Tensor(
[[-0.5644841 ]
 [-0.76016265]
 [ 0.30502525]], shape=(3, 1), dtype=float32)


- `uniform()` 함수: 지정된 구간 최솟값(`minval`)과 최댓값(`maxval`) 사이에서 균등분포를 따르는 항목들을 이용하여 상수 텐서를 생성한다.

In [6]:
x = tf.random.uniform(shape=(3, 1), minval=0., maxval=1.)
print(x)

tf.Tensor(
[[0.33661604]
 [0.09824598]
 [0.32487237]], shape=(3, 1), dtype=float32)


**수정 불가능성**

한 번 생성된 상수 텐서는 수정이 불가능하다.
예를 들어 인덱싱으로 항목을 수정할 수 없다.

```python
>>> x[0, 0] = 1.0
TypeError                                 Traceback (most recent call last)
<ipython-input-7-242a5d4d3c4a> in <module>
----> 1 x[0, 0] = 1.0

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object does not support item assignment
```

반면에 넘파이 어레이는 수정이 가능하다.

In [7]:
y = np.ones(shape=(2, 2))

y[0, 0] = 0.0
print(y)

[[0. 1.]
 [1. 1.]]


**텐서 항목의 자료형**

텐서 항목의 자료형은 `EagerTensor` 라는 텐서다.

In [8]:
type(x[0, 0])

tensorflow.python.framework.ops.EagerTensor

### 변수 텐서

`tf.Variaible` 객체는 항목을 수정할 수 있는 가변 텐서이다.
아래 코드는 정규 분포를 이용하여 (3. 1) 모양의 변수 텐서를 생성한다.

In [9]:
v = tf.Variable(initial_value=tf.random.normal(shape=(3, 1)))
print(v)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-1.3837979 ],
       [-0.23704937],
       [-0.9790895 ]], dtype=float32)>


**변수 텐서 교체**

`assign()` 메서드는 해당 텐서를 통채로 다른 텐서로 대체한다.

In [10]:
v.assign(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.]], dtype=float32)>

단, 대체하는 텐서의 모양(shape)이 기존 텐서의 모양과 동일해야 한다.

```python
>>> v.assign(tf.ones((3, 2)))
ValueError                                Traceback (most recent call last)
<ipython-input-13-e381ab0c94e6> in <module>
----> 1 v.assign(tf.ones((3, 2)))

~\anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py in assign(self, value, use_locking, name, read_value)
    886         else:
    887           tensor_name = " " + str(self.name)
--> 888         raise ValueError(
    889             ("Cannot assign to variable%s due to variable shape %s and value "
    890              "shape %s are incompatible") %

ValueError: Cannot assign to variable Variable:0 due to variable shape (3, 1) and value shape (3, 2) are incompatible
```

**변수 텐서 항목 변경**

변수 텐서의 특정 항목을 수정하려면 인덱싱과 `assign()` 메서드를 함께 사용한다.
이것이 가능한 이유는 텐서의 항목 또한 텐서이기 때문이다.

In [11]:
v[0, 0]

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [12]:
type(v[0, 0])

tensorflow.python.framework.ops.EagerTensor

상수 텐서의 경우처럼 각 항목은 `EagerTensor` 객체이지만 이번엔 항목 변환이 가능하다.
텐서플로우 내부에서 변수 텐서인 경우와 아닌 경우를 구분해서 `assign()` 메서드의 지원여부를
판단하는 것으로 보인다.

In [13]:
v[0, 0].assign(3.)

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[3.],
       [1.],
       [1.]], dtype=float32)>

* `assign_add()`와 `assign_sub()`

`assign_sub()` 함수는 텐서 자료형의 메서드이며 `-=` 연산자와 동일한 기능을 수행한다.
이외에 `assign_add()` 메서드는 `+=` 연산자의 기능을 갖는다.

In [14]:
v.assign_add(tf.ones((3, 1)))

<tf.Variable 'UnreadVariable' shape=(3, 1) dtype=float32, numpy=
array([[4.],
       [2.],
       [2.]], dtype=float32)>

## 랭크(차원)와 축

텐서는 사용되는 축<font size='2'>axis</font>의 개수에 따라
랭크<font size='2'>rank</font>가 정해진다.
랭크를 차원<font size='2'>dimension</font>이라 부르기도 한다.
텐서는 보통 `tf.constant()` 함수를 이용하여 생성한다.

**스칼라: 랭크-0 텐서**

스칼라는 하나의 정수 또는 하나의 부동소수점만 포함하며 따라서 축이 없다.
스칼라는 랭크-0 텐서라고도 불린다.
아래 `rank_0_tensor`는 int32 자료형인 정수 4를 포함한 스칼라를 가리킨다.

In [15]:
rank_0_tensor = tf.constant(4)
print(rank_0_tensor)

tf.Tensor(4, shape=(), dtype=int32)


**벡터: 랭크-1 텐서**

벡터는 하나의 축을 가지며 여러 개의 값을 포함한다.
아래 `rank_1_tensor`는 세 개의 부동소수점으로 구성된 벡터를 가리킨다.

In [16]:
rank_1_tensor = tf.constant([2.0, 3.0, 4.0])
print(rank_1_tensor)

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


**행렬: 랭크-2 텐서**

행렬에는 두 개의 축이 있다.
아래 `rank_2_tensor`는 랭크-2 텐서를 가리킨다.
또한 `dtype` 키워드 인자를 이용하여 자료형을 `float16`으로 지정하였다.
지정하지 않았다면 `rank_0_tensor`의 경우처럼 `float32`로
`dtype`으로 자동 지정되었을 것이다.
참고로 `float32`는 `float16` 보다 보다 큰 정수를 다룰 수 있으며
따라서 보다 많은 메모리와 계산 시간이 소비된다.

딥러닝 모델은 경우에 따라 매우 큰 텐서 연산을 실행해야 하기에
계산의 시간적, 공간적 효율성이 매우 중요하다.
따라서 반드시 필요한 만큼의 메모리를 확보하는
`dtype`을 지정하는 일이 중요하다.

In [17]:
rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)

tf.Tensor(
[[1. 2.]
 [3. 4.]
 [5. 6.]], shape=(3, 2), dtype=float16)


**스칼라, 벡터, 행렬의 모양**

텐서의 모양<font size='2'>shape</font>은 텐서의 축 별로 포함된
항목의 수에 대한 정보를 담는다.

- `rank_0_tensor` 스칼라의 모양: 스칼라는 축이 없기에 모양 정보도 없음

In [18]:
rank_0_tensor.shape

TensorShape([])

- `rank_1_tensor` 벡터의 모양: 0번 축에 3개의 항목

In [19]:
rank_1_tensor.shape

TensorShape([3])

- `rank_2_tensor` 행렬의 모양: 0번 축에 3개의 항목, 1번 축에 2개의 항목

In [20]:
rank_2_tensor.shape

TensorShape([3, 2])

그림으로 나타내면 다음과 같다.

<table>
<tr>
  <th>스칼라, 모양: <code>[]</code> </th>
  <th>벡터, 모양: <code>[3]</code> </th>
  <th>행렬, 모양: <code>[3, 2]</code> </th>
</tr>
<tr>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/scalar.png" alt="A scalar, the number 4"> </td>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/vector.png" alt="The line with 3 sections, each one containing a number.">   </td>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/matrix.png" alt="A 3x2 grid, with each cell containing a number.">   </td>
</tr>
</table>


**랭크-3 텐서**

세 개의 축을 사용한다.

In [21]:
rank_3_tensor = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]]
  ])

print(rank_3_tensor)

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)


모양은 다음과 같이 세 개의 축 각각에 사용된 항목의 수로 구성된
길이가 3인 리스트처럼 생겼다.

In [22]:
rank_3_tensor.shape

TensorShape([3, 2, 5])

랭크-3 텐서를 여러 가지 방식으로 시각화할 수 있다.

<table>
<tr>
  <th colspan="3">rank_3_tensor의 모양: <code>[3, 2, 5]</code> </th>
</tr>
<tr>
</tr>
<tr>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/3-axis_numpy.png">   </td>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/3-axis_front.png">   </td>
  <td>    <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/3-axis_block.png">   </td>
</tr>
</table>

## 넘파이 어레이로의 변환

`np.array()` 함수 또는 `tensor.numpy()` 메서드를 이용하여
`tf.tensor`를 `np.ndarray` 로 변환할 수 있다.

In [23]:
np.array(rank_2_tensor)

array([[1., 2.],
       [3., 4.],
       [5., 6.]], dtype=float16)

In [24]:
rank_2_tensor.numpy()

array([[1., 2.],
       [3., 4.],
       [5., 6.]], dtype=float16)

## 텐서 연산

텐서의 덧셈과 곱셈은 항목별로 이루어진다.

In [25]:
a = tf.constant([[1, 2],
                 [3, 4]])

In [26]:
b = tf.ones([2, 2], dtype='int32')

- 항목별 덧셈: `tf.add()` 함수 또는 `+` 연산자 이용

In [27]:
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [28]:
b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1]], dtype=int32)>

In [29]:
tf.add(a, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

In [30]:
a + b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

- 항목별 곱셈: `tf.multiply()` 함수 또는 `+` 연산자 이용

In [31]:
tf.multiply(a, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [32]:
a * b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

행렬(랭크-2 텐서) 연산은 항목별 연산과 다르다.

- 행렬 연산: `tf.matmul()` 함수 또는 `@` 연산자 이용

In [33]:
tf.matmul(a, b)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 3],
       [7, 7]], dtype=int32)>

In [34]:
a @ b

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3, 3],
       [7, 7]], dtype=int32)>

이외에도 텐서는 다양한 종류의 연산에 활용된다.

In [35]:
c = tf.constant([[4.0, 5.0], [10.0, 1.0]])

- 최대 항목 찾기

In [36]:
tf.reduce_max(c)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

- 최대 항목의 인덱스 확인

In [37]:
tf.math.argmax(c)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([1, 0])>

- `softmax()` 함수 적용

In [38]:
tf.nn.softmax(c)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2.6894143e-01, 7.3105860e-01],
       [9.9987662e-01, 1.2339458e-04]], dtype=float32)>

**텐서로의 자동 변환**

텐서플로우의 함수가 `tf.Tensor`를 인자로 입력받아야 하는데
인자가 `tf.Tensor` 자료형이 아니지만 텐서로 변환될 수 있으면
`tf.Tensor`로 자동 변환되어 처리된다.
이유는 내부적으로 `tf.convert_to_tensor()` 함수가
형변환을 실행하기 때문이다.
예를 들어, 리스트와 넘파이 어레이가 이에 해당한다.

In [39]:
tf.convert_to_tensor([1,2,3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [40]:
tf.reduce_max([1, 2, 3])

<tf.Tensor: shape=(), dtype=int32, numpy=3>

In [41]:
tf.math.argmax([1, 2, 3])

<tf.Tensor: shape=(), dtype=int64, numpy=2>

In [42]:
tf.nn.softmax(np.array([1.0, 12.0, 33.0]))

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([1.26641655e-14, 7.58256042e-10, 9.99999999e-01])>

## 텐서의 모양, 랭크, 축, 크기

다음 네 개념이 텐서의 모양을 설명한다.

- **모양**: 텐서에 사용된 각각의 축에 사용된 항목의 개수로 구성된 벡터
- **랭크** 또는 **차원**: 텐서에 사용된 축의 개수
    - 스칼라의 랭크는 0,
    - 벡터의 랭크는 1,
    - 행렬의 랭크는 2.
- **축**: 텐서 구성에 사용된 축
- **크기**: 텐서에 포함된 항목의 개수


예를 들어 아래 그림은 `rank_4_tensor`가 가리키는
랭크-4 텐서의 모양과 관련된 정보를 보여준다.
`tf.zeros()` 함수는 인자로 주어진 모양의 정보를 이용하여
모든 항목이 0으로 채워진 지정된 모양의 텐서를 생성한다.

In [43]:
rank_4_tensor = tf.zeros([3, 2, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(3, 2, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)>

<table>
<tr>
  <th colspan="2">랭크-4 텐서, 모양: <code>[3, 2, 4, 5]</code> </th>
</tr>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/shape.png" alt="A tensor shape is like a vector.">     </td>
<td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/4-axis_block.png" alt="A 4-axis tensor">   </td>
  </tr>
</table>


- 항목의 자료형

In [44]:
rank_4_tensor.dtype

tf.float32

- 텐서의 차원(랭크)

In [45]:
rank_4_tensor.ndim

4

- 텐서의 모양

In [46]:
rank_4_tensor.shape

TensorShape([3, 2, 4, 5])

- 0번 축에 사용되는 항목의 개수

In [47]:
rank_4_tensor.shape[0]

3

- -1번 축, 즉 마지막 축에 사용되는 항목의 개수

In [48]:
rank_4_tensor.shape[-1]

5

- 텐서의 크기

In [49]:
tf.size(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=120>

`ndim`과 `shape` 속성에 저장된 값은 `tf.Tensor` 객체가 아니다.
반면에  `tf.rank()` 함수 또는 `tf.shape()` 함수는 동일한 정보를
텐서로 반환한다.

In [50]:
tf.rank(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [51]:
tf.shape(rank_4_tensor)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([3, 2, 4, 5], dtype=int32)>

아래 그림은 랭크-4 텐서에 사용된 축 각각의 의미를 설명한다.
축은 텐서의 겉모양에서부터 출발하여 각각의 항목이 포함된 벡터까지
차례대로 배치<font size='2'>batch</font>,
높이<font size='2'>height</font>,
폭<font size='2'>width</font>,
특성<font size='2'>features</font>의
정보를 가리킨다.

<table>
<tr>
<th>축의 순서 이해</th>
</tr>
<tr>
    <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/shape2.png" alt="Keep track of what each axis is. A 4-axis tensor might be: Batch, Width, Height, Features">   </td>
</tr>
</table>

## 인덱싱/슬라이싱

예를 들어 아래 랭크-3 텐서를 이용한 인덱싱/슬라이싱 예를 하나 살펴보자.

In [52]:
rank_3_tensor

<tf.Tensor: shape=(3, 2, 5), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9]],

       [[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]]], dtype=int32)>

`rank_3_tensor` 텐서는 세 개의 축을 사용하는 랭크-3 텐서이며
모양은 다음과 같다.

In [53]:
rank_3_tensor.shape

TensorShape([3, 2, 5])

즉, 0번 축은 3 개의 항목을,
1번 축은 2 개의 항목을,
2번 축은 5 개의 항목을 각각 갖는다.

아래 코드는 0번 축과 1번 축의 모든 항목에 대해
2번 축의 4번 인덱스에 해당하는 값만 추출한다.

In [54]:
rank_3_tensor[:, :, 4]

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 4,  9],
       [14, 19],
       [24, 29]], dtype=int32)>

<table>
<tr>
<th colspan="2">배치에서 각 예의 모든 위치에서 마지막 특성 선택하기</th>
</tr>
<tr>
    <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/index1.png" alt="A 3x2x5 tensor with all the values at the index-4 of the last axis selected.">   </td>
      <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/index2.png" alt="The selected values packed into a 2-axis tensor.">   </td>
</tr>
</table>

## 모양 변환

주어진 텐서의 모양을 변환해서 새로운 모양의 텐서를 생성할 수 있다.
단, 새로운 텐서를 위해 메모리를 더 사용하는 것은 아니며
단지 주어진 텐서의 정보를 활용하기에 매우 빠르고 메모리
효율적으로 작동한다.
이렇게 새로운 텐서를 생성하는 방식을 **뷰**<font size='2'>view</font>라 부른다.


In [55]:
x = tf.constant([[1], [2], [3]])
x

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3]], dtype=int32)>

In [56]:
x.shape

TensorShape([3, 1])

In [57]:
reshaped = tf.reshape(x, [1, 3])
reshaped

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>

In [58]:
reshaped.shape

TensorShape([1, 3])

모양을 변환할 때 -1을 사용하면 그것은 해당 축의 항목의 개수는
다른 축의 항목의 개수에 맞춰서 자동으로 지정된다는 의미다.
이것이 가능한 이유는
텐서의 모양 변환을 통해 생성된 텐서는 기존의 텐서와
동일한 크기, 동일한 항목의 수를 가져야만 하기 때문이다.

예를 들어 아래 코드에서 사용된 -1은 3을 가리킨다.
이유는 총 세 개의 항목이 있는데 0번 축에 1개의 항목이 있어야 한다면
1번 축엔 세 개의 항목이 있어야 하기 때문이다.

In [59]:
reshaped = tf.reshape(x, [1, -1])
reshaped

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>

**항목 저장 순서**

임의의 텐서를 rank-1 텐서로 만들면 텐서의 항목들이 메모리상에서
저장되는 순서를 확인할 수 있다.

In [60]:
rank_3_tensor

<tf.Tensor: shape=(3, 2, 5), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9]],

       [[10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]]], dtype=int32)>

랭크-1 텐서로 변환하려면 모양을 `[-1]`로 지정하면 된다.
그러면 1개의 축만 사용하고, -1은 자동으로 전체 항목의 개수를
가기키기 때문이다.

In [61]:
tf.reshape(rank_3_tensor, [-1])

<tf.Tensor: shape=(30,), dtype=int32, numpy=
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], dtype=int32)>

**축의 순서를 고려하는 좋은 모양 변환**

`tf.reshape()` 함수는 앞서 확인한 항목들의 순서를 그대로 유지하며
단순히 인접한 두 개의 축을 합치거나, 하나의 축을 두 개의 축으로 나누거나,
또는 새로운 축을 추가하는 기능을 수행한다.

**주의사항:** 모양 변환은 축의 순서를 바꾸지는 않는다.
축의 순서를 바꾸러면 [`tf.transpose()`](https://www.tensorflow.org/api_docs/python/tf/transpose) 함수를 이용해야 하는데,
여기서는 자세히 다루지 않는다.

- `3x2x5` 텐서 => `(3x2)x5` 텐서

In [62]:
tf.reshape(rank_3_tensor, [3*2, 5])

<tf.Tensor: shape=(6, 5), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29]], dtype=int32)>

- `3x2x5` 텐서 => `3x(2x5)` 텐서

In [63]:
tf.reshape(rank_3_tensor, [3, -1])

<tf.Tensor: shape=(3, 10), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29]], dtype=int32)>

<table>
<th colspan="3">축의 순서를 반영하는 좋은 변환</th>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-before.png" alt="A 3x2x5 tensor">   </td>
  <td>   <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-good1.png" alt="The same data reshaped to (3x2)x5">   </td>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-good2.png" alt="The same data reshaped to 3x(2x5)">   </td>
</tr>
</table>


**축의 순서를 고려하지 않는 나쁜 모양 변환**

축의 순서를 고려하지 않으면
완전히 엉뚱한 텐서가 생성되거
모양 변환이 되지 않고 실행오류가 발생한다.

In [64]:
tf.reshape(rank_3_tensor, [2, 3, 5])

<tf.Tensor: shape=(2, 3, 5), dtype=int32, numpy=
array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14]],

       [[15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]]], dtype=int32)>

In [65]:
tf.reshape(rank_3_tensor, [5, 6])

<tf.Tensor: shape=(5, 6), dtype=int32, numpy=
array([[ 0,  1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10, 11],
       [12, 13, 14, 15, 16, 17],
       [18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29]], dtype=int32)>

In [66]:
try:
  tf.reshape(rank_3_tensor, [7, -1])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

InvalidArgumentError: {{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 30 values, but the requested shape requires a multiple of 7 [Op:Reshape]


아래 그림이 축의 순서를 고려하지 않은 잘못된 모양 변환의
결과를 보여준다.

<table>
<th colspan="3">몇 가지 잘못된 재구성</th>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-bad.png" alt="You can't reorder axes, use tf.transpose for that">   </td>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-bad4.png" alt="Anything that mixes the slices of data together is probably wrong.">   </td>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/reshape-bad2.png" alt="The new shape must fit exactly.">   </td>
</tr>
</table>

## 텐서 항목의 자료형(`dtype`) 변환

`tf.cast()` 함수를 이용하여 `dtype` 속성에 저장된
항목들의 자료형을 강제로 형변환하여
새로운 텐서를 생성할 수 있다.

- 예제: 64비트 부동소수점 항목으로 구성된 텐서 선언

In [67]:
the_f64_tensor = tf.constant([2.2, 3.3, 4.4], dtype=tf.float64)
the_f64_tensor

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2.2, 3.3, 4.4])>

- 16비트 부동소수점 항목으로 구성된 텐서로 형변환하여 텐서 선언

In [68]:
the_f16_tensor = tf.cast(the_f64_tensor, dtype=tf.float16)
the_f16_tensor

<tf.Tensor: shape=(3,), dtype=float16, numpy=array([2.2, 3.3, 4.4], dtype=float16)>

- 8비트 음이 아닌 정수<font size='2'>unsigned int</font>로 구성된 텐서로 형변환

In [69]:
the_u8_tensor = tf.cast(the_f16_tensor, dtype=tf.uint8)
the_u8_tensor

<tf.Tensor: shape=(3,), dtype=uint8, numpy=array([2, 3, 4], dtype=uint8)>

## 브로드캐스팅

예를 들어 벡터에 스칼라를 더하려 하면 스칼라가 벡터와 모양을 맞춘 후 항목별 덧셈이 실행된다. 이때 새로 생성되는 벡터의 항목은 모두 스칼라로 지정된다.
이처럼 두 연산자의 모양을 자동으로 맞춘 후 연산이 실행되도록 하는 것을
**브로드캐스팅**<font size='2'>broadcasting</font>이라 하며 넘파이 어레이 연산에 적용디는 그것과 동일하게 작동한다.

In [70]:
x = tf.constant([1, 2, 3])
y = tf.constant(2)
z = tf.constant([2, 2, 2])

다음 세 개의 연산 결과가 모두 동일하다.

In [71]:
print(tf.multiply(x, 2))
print(x * y)
print(x * z)

tf.Tensor([2 4 6], shape=(3,), dtype=int32)
tf.Tensor([2 4 6], shape=(3,), dtype=int32)
tf.Tensor([2 4 6], shape=(3,), dtype=int32)


벡터와 행렬의 연산도 유사하게 작동한다.

In [72]:
x = tf.reshape(x,[3,1])
x

<tf.Tensor: shape=(3, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3]], dtype=int32)>

In [73]:
y = tf.range(1, 5)
y

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([1, 2, 3, 4], dtype=int32)>

아래 곱셈 연산은 먼저 `x`와 `y`를 모두
`[3, 4]` 모양의 행렬로 브로드캐스팅을 한 다음에
항목별 곱셈을 실행한다.

In [74]:
x * y

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[ 1,  2,  3,  4],
       [ 2,  4,  6,  8],
       [ 3,  6,  9, 12]], dtype=int32)>

<table>
<tr>
  <th>추가 시 브로드캐스팅: <code>[1, 4]</code>와 <code>[3, 1]</code>의 곱하기는 <code>[3,4]</code>입니다.</th>
</tr>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/broadcasting.png" alt="Adding a 3x1 matrix to a 4x1 matrix results in a 3x4 matrix">   </td>
</tr>
</table>


실제로 내부적으로 아래 연산과 동일하게 작동한다.

In [75]:
x_stretch = tf.constant([[1, 1, 1, 1],
                         [2, 2, 2, 2],
                         [3, 3, 3, 3]])

y_stretch = tf.constant([[1, 2, 3, 4],
                         [1, 2, 3, 4],
                         [1, 2, 3, 4]])

print(x_stretch * y_stretch)

tf.Tensor(
[[ 1  2  3  4]
 [ 2  4  6  8]
 [ 3  6  9 12]], shape=(3, 4), dtype=int32)


## 다양한 종류의 텐서

### 비정형 텐서

**비정형 텐서**<font size='2'>raggedn tensor</font>는
벡터의 길이가 일정하지 않은 축이 사용되는 텐서를 가리킨다.
`tf.Tensor` 자료형으로는 선언할 수 없으며
대신 `tf.ragged.RaggedTensor` 자료형으로 선언한다.

예를 들어, 아래 그림은 0번 축의 길이는 4이지만 1번 축의
길이가 일정하지 않는 비정형 텐서를 보여준다.
아래 비정형 텐서의 모양은 `[4, None]`으로 표시한다.
`None`은 해당 축에 포함된 벡터들의 길이가 일정하지 않음을 의미한다.

<table>
<tr>
  <th><code>[4, None]</code> 모양의 비정형 텐서 </th>
</tr>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/ragged.png" alt="A 2-axis ragged tensor, each row can have a different length.">   </td>
</tr>
</table>

위 그림 모양의 비정형 텐서를 선언해 보자.
이를 위해 아래 리스트를 이용한다.

In [76]:
ragged_list = [
    [0, 1, 2, 3],
    [4, 5],
    [6, 7, 8],
    [9]]

정규 텐서로는 선언할 수 없다.

In [77]:
try:
  tensor = tf.constant(ragged_list)
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Can't convert non-rectangular Python sequence to Tensor.


대신 `tf.ragged.constant`를 사용하여 `tf.RaggedTensor`로 선언할 수 있다.

In [78]:
ragged_tensor = tf.ragged.constant(ragged_list)
print(ragged_tensor)

<tf.RaggedTensor [[0, 1, 2, 3], [4, 5], [6, 7, 8], [9]]>


`tf.RaggedTensor`의 모양에서 축의 길이를 알 수 없는 축은 `None`으로 나타낸다.

In [79]:
print(ragged_tensor.shape)

(4, None)


### 희소 텐서

텐서의 크기가 매우 큰 반면에 0이 아닌 항목의 개수가 상대적으로
적을 때 희소 텐서<font size='2'>sparse tensor</font>를
사용하면 메모리를 보다 효율적으로 활용할 수 있다.
희소 텐서는 `tf.sparse.SparseTensor` 자료형이 지원한다.

아래 그림은 `[3, 4]` 모양의 텐서 단 두 개의 값을 제외한 나머지는
0인 희소 텐서를 보여준다.

<table>
<tr>
  <th>`tf.SparseTensor`, 모양: <code>[3, 4]</code> </th>
</tr>
<tr>
  <td> <img src="https://raw.githubusercontent.com/tensorflow/docs/master/site/en/guide/images/tensor/sparse.png" alt="An 3x4 grid, with values in only two of the cells.">   </td>
</tr>
</table>

아래 `sparse_tensor` 변수는 위 그림으로 표현된
희소 텐서를 가리킨다.
`tf.sparse.SparseTensor` 객체를 선언할 때
다음 세 개의 인자를 지정한다.

- `indices`: 0이 아닌 항목들의 인덱스로 구성된 리스트
- `values`: 지정된 인덱스에 위치하는 항목들의 리스트
- `dense_shape`: 텐서의 모양

In [80]:
sparse_tensor = tf.sparse.SparseTensor(indices=[[0, 0], [1, 2]],
                                       values=[1, 2],
                                       dense_shape=[3, 4])
sparse_tensor

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))

**밀집 텐서 대 희소 텐서**

일반적인 텐서를 **밀집 텐서**<font size='2'>dense tensor</font>라
부른다.
텐서플로우는 내부적으로 희소 텐서와 밀집 텐서를 모두 다룰 수 있으며
필요에 따라 변환해서 사용한다.

희소 텐서를 밀집 텐서로 직접 변환하려면
`tf.sparse.to_dense()` 함수를 이용하면 된다.

In [81]:
dense_tensor = tf.sparse.to_dense(sparse_tensor)
dense_tensor

<tf.Tensor: shape=(3, 4), dtype=int32, numpy=
array([[1, 0, 0, 0],
       [0, 0, 2, 0],
       [0, 0, 0, 0]], dtype=int32)>

반면에 `tf.sparse.from_dense()` 함수는 밀집 텐서를 희소 텐서로 변환한다.

In [82]:
tf.sparse.from_dense(dense_tensor)

SparseTensor(indices=tf.Tensor(
[[0 0]
 [1 2]], shape=(2, 2), dtype=int64), values=tf.Tensor([1 2], shape=(2,), dtype=int32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))